# **Appendix A: Assumption 1 on 2x2x2 Rubik's Cube**
---
[![GitHub Repository](https://img.shields.io/badge/-Repository-2dba4e?logo=github&style=flat)](https://github.com/kyo-takano/EfficientCube)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kyo-takano/EfficientCube/blob/main/notebooks/EfficientCube_Assumption_1.ipynb)
[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/kyo-takano/EfficientCube/blob/main/notebooks/EfficientCube_Assumption_1.ipynb)

This notebook is intended to replicate Appendix A of the paper

Computing all possible solutions except for self-canceling permutations, we obtain the frequency ranks of optimal moves at all nodes.\
As in the paper, the action space is set to HTM, which has 9 possible moves per state. In this configuration, God's Number is 11 moves.\
We use the built-in module [`fractions`](https://docs.python.org/3/library/fractions.html) to compute exact probabilities.

## Setup

In [ ]:
import time
import itertools
import numpy as np
from fractions import Fraction
from tqdm import tqdm

## 2x2x2 Rubik's Cube

In [ ]:
#@title
import random
import numpy as np

class Cube2(object):
    def __init__(self, METRIC="HTM"):
        self.DTYPE = np.int64
        self.METRIC = METRIC

        # define state and goal
        self.reset()  # state
        self.goal = np.arange(0, 4*6, dtype=self.DTYPE) // 4

        # define moves
        ## faces and turns
        """
            As we compute 'relative' transition of states, we turn U (Up), R (Right), F (Front) faces only.
            Accordingly, the Left-Down-Back piece is fixed.
        """
        faces = ['U', 'R', 'F']
        degrees = ['', "'", '2'] # [90 degrees clockwise, 90 degrees counter-clockwise, 180 degrees]

        # cartesian product
        self.moves = [f"{f}{n}" for f in faces for n in degrees]
        # two subsequent moves on a same face are unnecessary.
        self.moves_subsequent = {
            m: [v for v in self.moves if v[0] != m[0]] for m in self.moves
        }
        self.__vectorize_moves()

    def reset(self):
        # self.state = self.goal.copy()
        self.state = np.arange(0, 4*6, dtype=self.DTYPE) // 4

    def is_solved(self):
        return np.all(self.state == self.goal)

    def state_to_batch(self):
        return np.expand_dims(self.state, axis=0)

    def finger(self, move):
        self.state[self.sticker_target[move]
                   ] = self.state[self.sticker_source[move]]

    def finger_ix(self, ix):
        self.state[self.sticker_target_ix[ix]
                   ] = self.state[self.sticker_source_ix[ix]]

    def __vectorize_moves(self):
        '''
            This method defines "self.sticker_target" and "self.sticker_source" to manage sticker colors (target is replaced by source).
            They define indices of target and source stickers so that the moves can be vectorized.

            colors:
                    0 0
                    0 0
                2 2 5 5 3 3 4 4
                2 2 5 5 3 3 4 4
                    1 1
                    1 1
            order of stickers on each face:
                 1  3
                [0] 2

            indices of state (each starting with 4*(n-1)):
                         1   3
                        [0]  2
                 9  11  21  23   13  15  17  19
                [8] 10 [20] 22  [12] 14 [16] 18
                         5   7
                        [4]  6
        '''
        self.sticker_target, self.sticker_source = dict(), dict()

        self.sticker_replacement = {
            # Sticker A is replaced by another sticker at index B -> A:B
            'U': {
                0: 2,
                1: 0,
                2: 3,
                3: 1,
                9: 21,
                11: 23,
                13: 17,
                15: 19,
                17: 9,
                19: 11,
                21: 13,
                23: 15
            },
            'R': {
                2: 22,
                3: 23,
                7: 16,
                6: 17,
                12: 14,
                13: 12,
                14: 15,
                15: 13,
                16: 3,
                17: 2,
                22: 6,
                23: 7
            },
            'F': {
                0: 10,
                2: 11,
                5: 12,
                7: 13,
                10: 7,
                11: 5,
                12: 2,
                13: 0,
                20: 22,
                21: 20,
                22: 23,
                23: 21
            }
        }
        for m in self.moves:
            if len(m) == 1:
                assert m in self.sticker_replacement
            else:
                if "'" in m:
                    self.sticker_replacement[m] = {
                        v: k for k, v in self.sticker_replacement[m[0]].items()}
                elif "2" in m:
                    self.sticker_replacement[m] = {
                        k: self.sticker_replacement[m[0]][v] for k, v in self.sticker_replacement[m[0]].items()}
                else:
                    raise

            self.sticker_target[m] = list(self.sticker_replacement[m].keys())
            self.sticker_source[m] = list(self.sticker_replacement[m].values())

            for i, idx in enumerate(self.sticker_target[m]):
                assert self.sticker_replacement[m][idx] == self.sticker_source[m][i]

        self.sticker_source_ix = {
            ix: self.sticker_source[m] for ix, m in enumerate(self.moves)}
        self.sticker_target_ix = {
            ix: self.sticker_target[m] for ix, m in enumerate(self.moves)}


## Experiment: Exhausting all the possible path from the goal

In [ ]:
# setup
"""
    p: path
    m: move
    S: State
"""

# get env and configurations
cube = Cube2()
moves = cube.moves
len_moves = len(moves)
MAX_DEPTH = 11


## if exclude self-canceling moves
skip_redundant = True
## pre-compute probability of a path by its depth
if skip_redundant:
    depth_to_prob = {}
    for depth in range(1, MAX_DEPTH+1):
        C = len_moves
        if depth > 1:
            C *= (len_moves-3)**(depth-1)
        depth_to_prob[depth] = Fraction(1, C)
else:
    depth_to_prob = {depth:Fraction(1, len_moves**depth) for depth in range(1,MAX_DEPTH+1)}        

# hash table for 'ootimal moves' and another for 'cumulative probabilities of moves (whether or not optimal)'.
state2opt = {}
state2prob = {}

# MAIN
for depth in range(1, MAX_DEPTH+1):
    time_0 = time.time()
    counter = 0
    g = itertools.product(moves, repeat=depth)
    for p in g:        
        if skip_redundant:
            if depth > 1:
                bool_continue = False
                for i, m in enumerate(p[:-1]):
                    if m[0] == p[i+1][0]:
                        bool_continue = True
                        break
                if bool_continue:
                    continue
        
        # get the indices of moves
        p = [moves.index(m) for m in p]
        # get the scrambled state
        cube.reset()
        _ = [cube.finger_ix(m) for m in p]

        # obtain state as bytes (for memory efficiency)
        S = cube.state.tobytes()
        # get the move first move of the path. Because the transition is relative, it also can be the 'last' move instead.
        m = p[0]

        state2prob.setdefault(S, [Fraction()]*len_moves)[m] += depth_to_prob[depth]
        if S not in state2opt: # meaning it's the first & optimal move
            state2opt.setdefault(S, []).append((depth, m))
        else:
            if state2opt[S][0][0] == depth: # even if not the first, it still might be optimal (at the same depth as the existing optimal move)
                state2opt[S].append((depth, m))

        counter += 1

    t_delta = time.time() - time_0
    print(f'[depth:{depth}] -> {round(t_delta)}s\t#states:{len(state2prob)}\t#paths:{counter}')
    assert counter == Fraction(1, depth_to_prob[depth])


In [ ]:
# aggregate results
rank_highest, rank_lowest = [0]*len_moves, [0]*len_moves

for s, p_by_mix in tqdm(state2prob.items(), total=3674160):
    optimal_solutions = state2opt[s]
    moves_most_optimal = [opt[-1] for opt in optimal_solutions]
    rank_p_by_p = {p: r for r, p in enumerate(sorted(set(p_by_mix), reverse=True), 1)}
    rank_p_by_mix = {k: rank_p_by_p[v] for k, v in enumerate(p_by_mix)}
    rank_best = min([rank_p_by_mix[m] for m in moves_most_optimal])
    rank_worst = max([rank_p_by_mix[m] for m in moves_most_optimal])
    rank_highest[rank_best-1] += 1
    rank_lowest[rank_worst-1] += 1

print(f"rank_highest:\t{rank_highest}")
print(f"rank_lowest:\t{rank_lowest}")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.bar(np.arange(len(rank_highest))+1 - 0.2, rank_highest, 0.4, label='Highest', color='#00aeff')
ax.bar(np.arange(len(rank_lowest))+1 + 0.2, rank_lowest, 0.4, label='Lowest', color='#ff008c')
ax.set_xlabel('Rank of optimal moves')
ax.set_ylabel('Number of states')
plt.xticks([i for i in range(1,len(rank_highest)+1)])
ax.set_xlim(0,len(rank_highest)+0.5)
ax.set_ylim(0,)
ax.legend()
plt.show()